In [1]:
# conda: xesmfTools
# xarray OpenDap example
# adapted from: http://xarray.pydata.org/en/stable/examples/ROMS_ocean_model.html

# Partially works

import xarray as xr
import numpy as np

In [2]:
ds = xr.open_dataset('http://barataria.tamu.edu:8080/thredds/dodsC/NcML/txla_hindcast_agg', chunks={'ocean_time': 1})

OSError: [Errno -68] NetCDF: I/O failure: b'http://barataria.tamu.edu:8080/thredds/dodsC/NcML/txla_hindcast_agg'

In [ ]:
if ds.Vtransform == 1:
    Zo_rho = ds.hc * (ds.s_rho - ds.Cs_r) + ds.Cs_r * ds.h
    z_rho = Zo_rho + ds.zeta * (1 + Zo_rho/ds.h)
elif ds.Vtransform == 2:
    Zo_rho = (ds.hc * ds.s_rho + ds.Cs_r * ds.h) / (ds.hc + ds.h)
    z_rho = ds.zeta + (ds.zeta + ds.h) * Zo_rho

ds.coords['z_rho'] = z_rho.transpose()   # needing transpose seems to be an xarray bug
ds.salt

In [ ]:
ds.salt.isel(xi_rho=50, ocean_time=0).plot()

In [ ]:
section = ds.salt.isel(xi_rho=50, eta_rho=slice(0, 167), ocean_time=0)
section

In [ ]:
section.plot(x='lon_rho', y='z_rho', figsize=(15, 6), clim=(25, 35))
plt.ylim([-100, 1])

In [ ]:
proj = ccrs.LambertConformal(central_longitude=-92, central_latitude=29)
fig = plt.figure(figsize=(15, 5))
ax = plt.axes(projection=proj)
ds.salt.isel(s_rho=-1, ocean_time=0).plot(x='lon_rho', y='lat_rho',
                                          transform=ccrs.PlateCarree())

coast_10m = cfeature.NaturalEarthFeature('physical', 'land', '10m',
                                        edgecolor='k', facecolor='0.8')
ax.add_feature(coast_10m)